In [ ]:
!pip install sentencepiece
import os
import re
import json
import h5py
import logging
import numpy as np
import pandas as pd
import sentencepiece as spm
from tqdm import tqdm


# RAW_DATA_DIR = "/content/drive/MyDrive/colab_data/cj_data/data/bert/naver_10_data.pkl" # 카카오에서 다운로드 받은 데이터의 디렉터리
PROCESSED_DATA_DIR = '/content/drive/MyDrive/colab_data/cj_data/data/bert/preprocess' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR =os.path.join(PROCESSED_DATA_DIR, 'vocab') # 전처리에 사용될 사전 파일이 저장될 디렉터리


     |████████████████████████████████| 1.2 MB 12.4 MB/s 


In [ ]:
# 문장의 특수기호 제거 함수
def remove_special_characters(sentence, lower=True):
    # 특수기호를 나열한 패턴 문자열을 컴파일하여 패턴 객체를 얻는다.
    p = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')    

    sentence = p.sub(' ', sentence) # 패턴 객체로 sentence 내의 특수기호를 공백문자로 치환한다.
    sentence = ' '.join(sentence.split()) # sentence 내의 두개 이상 연속된 빈공백들을 하나의 빈공백으로 만든다.
    if lower:
        sentence = sentence.lower()
    return sentence

# sentencepiece 모델을 학습시키는 함수이다.
def train_spm(txt_path, spm_path,
              vocab_size=32000, input_sentence_size=1000000):  
    # input_sentence_size: 개수 만큼만 학습데이터로 사용된다.
    # vocab_size: 사전 크기
    spm.SentencePieceTrainer.Train(
        f' --input={txt_path} --model_type=bpe'
        f' --model_prefix={spm_path} --vocab_size={vocab_size}'
        f' --input_sentence_size={input_sentence_size}'
        f' --shuffle_input_sentence=true'
        f' --minloglevel=2'
    )

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_train_v1.csv")
df_test = pd.read_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_test_v1.csv")

df_train = data_delete(df_train)
df_train = data_change(df_train)

df_test = data_delete(df_test)
df_test = data_change(df_test)

data_df = pd.concat([df_train, df_test])
data_df

,Unnamed: 0,name,middle_classification
0,88430,파충류 용품 크레스티드게코 크레 붙이류 및 절지류 아크릴 사육장,생활/건강 반려동물
1,60470,자전거 스텐드 펌프 슈레더 프레스타 호환 공기주입기 키후,스포츠/레저 자전거
2,35466,"생활공작소 일회용 수세미, 그레이, 2개",생활/건강 수납/정리용품
3,341,"뷰티레시피 스노우화이트 멜라 스탑 화이트닝 크림, 80ml, 1개",화장품/미용 스킨케어
4,8027,타이타우 익스클루시브 다크초콜릿 99 카카오99 초콜렛 린트 간식 사탕,식품 과자
...,...,...,...
21995,98387,핸드폰 스마트폰 핑거 고리 스트랩 휴대폰줄 조절 가능한 휴대 전화 손목 가제트,디지털/가전 휴대폰액세서리
21996,48952,"Apple 2021 아이맥 24, 실버, M1 8-Core, 8GB, SSD 256...",디지털/가전 PC
21997,15211,"스타벅스 하우스 블렌드 리치 & 라이블리 커피 미디엄, 340g, 홀빈(분쇄안함)",식품 음료
21998,61399,피싱이오 수갑형 선상받침대 (방아쇠 포함) 선상거치대 선상받침틀,스포츠/레저 낚시


In [ ]:
data_df = data_df[['name', 'middle_classification']]

data_df

,name,middle_classification
0,파충류 용품 크레스티드게코 크레 붙이류 및 절지류 아크릴 사육장,생활/건강 반려동물
1,자전거 스텐드 펌프 슈레더 프레스타 호환 공기주입기 키후,스포츠/레저 자전거
2,"생활공작소 일회용 수세미, 그레이, 2개",생활/건강 수납/정리용품
3,"뷰티레시피 스노우화이트 멜라 스탑 화이트닝 크림, 80ml, 1개",화장품/미용 스킨케어
4,타이타우 익스클루시브 다크초콜릿 99 카카오99 초콜렛 린트 간식 사탕,식품 과자
...,...,...
21995,핸드폰 스마트폰 핑거 고리 스트랩 휴대폰줄 조절 가능한 휴대 전화 손목 가제트,디지털/가전 휴대폰액세서리
21996,"Apple 2021 아이맥 24, 실버, M1 8-Core, 8GB, SSD 256...",디지털/가전 PC
21997,"스타벅스 하우스 블렌드 리치 & 라이블리 커피 미디엄, 340g, 홀빈(분쇄안함)",식품 음료
21998,피싱이오 수갑형 선상받침대 (방아쇠 포함) 선상거치대 선상받침틀,스포츠/레저 낚시


In [ ]:

data_df['name'] = data_df['name'].map(remove_special_characters)

data_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,middle_classification
0,파충류 용품 크레스티드게코 크레 붙이류 및 절지류 아크릴 사육장,생활/건강 반려동물
1,자전거 스텐드 펌프 슈레더 프레스타 호환 공기주입기 키후,스포츠/레저 자전거
2,생활공작소 일회용 수세미 그레이 2개,생활/건강 수납/정리용품
3,뷰티레시피 스노우화이트 멜라 스탑 화이트닝 크림 80ml 1개,화장품/미용 스킨케어
4,타이타우 익스클루시브 다크초콜릿 99 카카오99 초콜렛 린트 간식 사탕,식품 과자
...,...,...
21995,핸드폰 스마트폰 핑거 고리 스트랩 휴대폰줄 조절 가능한 휴대 전화 손목 가제트,디지털/가전 휴대폰액세서리
21996,apple 2021 아이맥 24 실버 m1 8 core 8gb ssd 256gb a...,디지털/가전 PC
21997,스타벅스 하우스 블렌드 리치 라이블리 커피 미디엄 340g 홀빈 분쇄안함,식품 음료
21998,피싱이오 수갑형 선상받침대 방아쇠 포함 선상거치대 선상받침틀,스포츠/레저 낚시


In [ ]:
PROCESSED_DATA_DIR = '/content/drive/MyDrive/colab_data/cj_data/data/bert/' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR =os.path.join(PROCESSED_DATA_DIR, 'vocab') # 전처리에 사용될 사전 파일이 저장될 디렉터리

# product 칼럼의 상품명을 product.txt 파일명으로 저장한다.
with open(os.path.join(VOCAB_DIR, 'product_name_v1.txt'), 'w', encoding='utf-8') as f:
    f.write(data_df['name'].str.cat(sep='\n'))

In [ ]:
train_spm(txt_path=os.path.join(VOCAB_DIR, 'product_name_v1.txt'), 
              spm_path=os.path.join(VOCAB_DIR, 'spm_v1')) # spm 접두어

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load(os.path.join(VOCAB_DIR, 'spm_v1.model'))

True

In [ ]:
# product 칼럼의 상품명을 분절한 결과를 tokenized_product 칼럼에 저장한다.
data_df['tokens'] = data_df['name'].map(lambda x: " ".join(sp.EncodeAsPieces(x)))


In [ ]:
df_train['name'] = df_train['name'].map(remove_special_characters)
df_test['name'] = df_test['name'].map(remove_special_characters)


df_train['tokens'] = df_train['name'].map(lambda x: " ".join(sp.EncodeAsPieces(x)) )
df_test['tokens'] = df_test['name'].map(lambda x: " ".join(sp.EncodeAsPieces(x)) )


In [ ]:
data_df['대분류'] = data_df.middle_classification.map(lambda x: x.split(' ')[0])
data_df['중분류'] = data_df.middle_classification.map(lambda x: x.split(' ')[-1])

data_df['main_ctg'] = data_df.대분류.astype('category').cat.codes
data_df['midd_ctg'] = data_df.중분류.astype('category').cat.codes


# make ctg code mapping table
ctg_dict = data_df[['middle_classification', 'main_ctg', 'midd_ctg']].drop_duplicates()


In [ ]:
df_train


df_train['main_ctg'] = df_train.middle_classification.map(lambda x:ctg_dict[ctg_dict.middle_classification == x].values[0][1])
df_train['midd_ctg'] = df_train.middle_classification.map(lambda x:ctg_dict[ctg_dict.middle_classification == x].values[0][2])

df_test['main_ctg'] = df_test.middle_classification.map(lambda x:ctg_dict[ctg_dict.middle_classification == x].values[0][1])
df_test['midd_ctg'] = df_test.middle_classification.map(lambda x:ctg_dict[ctg_dict.middle_classification == x].values[0][2])

df_train

,Unnamed: 0,name,middle_classification,tokens,main_ctg,midd_ctg
0,88430,파충류 용품 크레스티드게코 크레 붙이류 및 절지류 아크릴 사육장,생활/건강 반려동물,▁파충류 ▁용품 ▁크레스티드게코 ▁크레 ▁붙이 류 ▁및 ▁절 지 류 ▁아크릴 ▁사육장,3,49
1,60470,자전거 스텐드 펌프 슈레더 프레스타 호환 공기주입기 키후,스포츠/레저 자전거,▁자전거 ▁스텐드 ▁펌프 ▁슈 레더 ▁프레 스타 ▁호환 ▁공기주입기 ▁키 후,4,131
2,35466,생활공작소 일회용 수세미 그레이 2개,생활/건강 수납/정리용품,▁생활공작소 ▁일회용 ▁수세미 ▁그레이 ▁2 개,3,69
3,341,뷰티레시피 스노우화이트 멜라 스탑 화이트닝 크림 80ml 1개,화장품/미용 스킨케어,▁뷰티 레시피 ▁스노우화이트 ▁멜라 ▁스탑 ▁화이트닝 ▁크림 ▁80 ml ▁1 개,10,82
4,8027,타이타우 익스클루시브 다크초콜릿 99 카카오99 초콜렛 린트 간식 사탕,식품 과자,▁타이 타 우 ▁익스 클 루 시브 ▁다크초콜릿 ▁99 ▁카카오 99 ▁초콜렛 ▁린 ...,5,13
...,...,...,...,...,...,...
200471,20441,메디탑 소독용 에탄올 83 1l,생활/건강 의료용품,▁메디 탑 ▁소독용 ▁에탄올 ▁83 ▁1 l,3,120
200472,66795,피엘라벤 하이 코스트 힙 팩 23223,패션잡화 남성가방,▁피 엘라 벤 ▁하이 ▁코스트 ▁힙 ▁팩 ▁23 223,9,23
200473,3482,미쟝센 쉽고 빠른 거품 염색 새치전용 5n 갈색 1개,화장품/미용 헤어케어,▁미쟝센 ▁쉽고 ▁빠른 ▁거품 ▁염색 ▁새치전용 ▁5 n ▁갈색 ▁1 개,10,163
200474,26086,르번 캐치티니핑 손잡이 스텐컵 혼합색상 1개,출산/육아 이유식용품,▁르번 ▁캐치티니핑 ▁손잡이 ▁스텐컵 ▁혼합색상 ▁1 개,7,123


In [ ]:
ctg_dict

,middle_classification,main_ctg,midd_ctg
0,생활/건강 반려동물,3,49
1,스포츠/레저 자전거,4,131
2,생활/건강 수납/정리용품,3,69
3,화장품/미용 스킨케어,10,82
4,식품 과자,5,13
...,...,...,...
1793,출산/육아 카시트,7,152
3006,식품 냉동/간편조리식품,5,28
3171,패션의류 임부복,8,127
3804,디지털/가전 액세서리,2,94


In [ ]:
df_train.to_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_train_v1_tokenized.csv", index = False)
df_test.to_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_test_v1_tokenized.csv", index = False)


In [ ]:
df_train

,Unnamed: 0,name,middle_classification,tokens,main_ctg,midd_ctg
0,88430,파충류 용품 크레스티드게코 크레 붙이류 및 절지류 아크릴 사육장,생활/건강 반려동물,▁파충류 ▁용품 ▁크레스티드게코 ▁크레 ▁붙이 류 ▁및 ▁절 지 류 ▁아크릴 ▁사육장,3,49
1,60470,자전거 스텐드 펌프 슈레더 프레스타 호환 공기주입기 키후,스포츠/레저 자전거,▁자전거 ▁스텐드 ▁펌프 ▁슈 레더 ▁프레 스타 ▁호환 ▁공기주입기 ▁키 후,4,131
2,35466,생활공작소 일회용 수세미 그레이 2개,생활/건강 수납/정리용품,▁생활공작소 ▁일회용 ▁수세미 ▁그레이 ▁2 개,3,69
3,341,뷰티레시피 스노우화이트 멜라 스탑 화이트닝 크림 80ml 1개,화장품/미용 스킨케어,▁뷰티 레시피 ▁스노우화이트 ▁멜라 ▁스탑 ▁화이트닝 ▁크림 ▁80 ml ▁1 개,10,82
4,8027,타이타우 익스클루시브 다크초콜릿 99 카카오99 초콜렛 린트 간식 사탕,식품 과자,▁타이 타 우 ▁익스 클 루 시브 ▁다크초콜릿 ▁99 ▁카카오 99 ▁초콜렛 ▁린 ...,5,13
...,...,...,...,...,...,...
200471,20441,메디탑 소독용 에탄올 83 1l,생활/건강 의료용품,▁메디 탑 ▁소독용 ▁에탄올 ▁83 ▁1 l,3,120
200472,66795,피엘라벤 하이 코스트 힙 팩 23223,패션잡화 남성가방,▁피 엘라 벤 ▁하이 ▁코스트 ▁힙 ▁팩 ▁23 223,9,23
200473,3482,미쟝센 쉽고 빠른 거품 염색 새치전용 5n 갈색 1개,화장품/미용 헤어케어,▁미쟝센 ▁쉽고 ▁빠른 ▁거품 ▁염색 ▁새치전용 ▁5 n ▁갈색 ▁1 개,10,163
200474,26086,르번 캐치티니핑 손잡이 스텐컵 혼합색상 1개,출산/육아 이유식용품,▁르번 ▁캐치티니핑 ▁손잡이 ▁스텐컵 ▁혼합색상 ▁1 개,7,123
